In [1]:
import urlexpander
import pandas as pd
import numpy as np
from lxml import etree
from pandas.core.frame import DataFrame
from tqdm import tqdm
import requests
import bs4
import math
import pymssql
import xml.etree.ElementTree as ET
import os, time, random
import threading
from random import randint 
import csv

In [2]:
def get_header( ):
    referer = 'https://www.eurekalert.org/'
    headers = {

        'referer': referer,
        'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36',
        'content-type': 'application/x-www-form-urlencoded',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',

        }
    return headers


def get_data(url):
    '''
    功能：访问 url 的网页，获取网页内容并返回
    参数：
        url ：目标网页的 url
    返回：目标网页的 html 内容
    '''
    headers = get_header( )
    
    try:
        r = requests.get(url, headers=headers)
        #r.encoding = "utf-8-sig"
        r.raise_for_status()
        return r.text
    
    #except requests.HTTPError as e:
       # print(e)
        #print("HTTPError")
    #except requests.RequestException as e:
       # print(e)
    except:
        null = 0
        return null

In [3]:
tree = ET.parse('C:/Users/zhangz10/ER/sitemap.xml')
root = tree.getroot()
urlmaps = []
for elem in root:
    li = []
    for subelem in elem:
        li.append(subelem.text)
    urlmaps.append(li[0])
urlmaps= urlmaps[0:len(urlmaps)-1]

In [4]:
urls = []
for url in urlmaps:
    r = requests.get(url)
    root = ET.fromstring(r.content)
    for elem in root:
        li = []
        for subelem in elem:
            li.append(subelem.text)
        urls.append(li[0])

In [333]:
d = {}
for i in urls:
    # get article id
    j=i[i.rindex('ses')+4:]
    d[j] = i
df = pd.DataFrame(d.items())
df.columns = ['euid', 'url']

In [427]:
dfurl.to_csv(r'eurekalert_2103-2303.csv'
          #, header=None
          , index=None
          , sep=';'
          , encoding='utf-8')

In [424]:
#eurekalert url data
f = open('dataset.txt')
lines = f.readlines()
l = []
for line in lines:
    li = []
    euid = line.replace('\n','').split('/')[2]
    url = 'http://www.'+ line.replace('\n','')
    li = [euid,url]
    l.append(li)
dfurl = pd.DataFrame(l)
dfurl.columns = ['euid','url']
dfurl = pd.concat([dfurl,df],ignore_index = True)

In [4]:
def get_eurekalert(euid,url):
    r = requests.get(url)
    html = r.content
    soup = bs4.BeautifulSoup(html)
    parse_html = etree.HTML(html)
    
#metadata
    doi = title= description= keywords= funder= journal= category= institution= meeting= date= original_source= None None
    
    title = soup.find_all('meta', property="og:title")[0]['content']
    description = soup.find_all('meta', property="og:description")[0]['content']
    
    k1 = soup.find_all('span',class_ ="ea-keyword__short")
    k2 = soup.find_all('span',class_ ="ea-keyword__path" )
    if k1:
        keywords1 =''
        keyword = []
        for i in range(len(k1)):
            keyword.append(k1[i].string)
            if k2[i]:
                if i == len(k1)-1:
                    keyword.append(k2[i].string)
                else:
                    keyword.append(k2[i].string+',') 
        for j in keyword:
            keywords1 += str(j)
        keywords = keywords1
    
    jf1 = soup.find_all('dt')
    jf2 = soup.find_all('dd')
    for i in range(len(jf1)):
        if 'DOI' in jf1[i]:
            doi = jf2[i].string
        if 'Journal' in jf1[i]:
            journal = jf2[i].string
        if 'Funder' in jf1[i]:
            funder = jf2[i].string.replace('\t','').replace('\n','').replace('  ','')
        if 'Meeting' in jf1[i]:
            meeting = jf2[i].string.replace('\t','').replace('\n','').replace('  ','')
            
    category = soup.find_all('a',style="color:red;")[0].string  
    institution =soup.find_all('p',class_="meta_institute")[0].string
    #post time
    date = soup.find_all('time')[0].string.replace(' ','').replace('\n','')
    if date == '':
        date = parse_html.xpath('//*[@id="ahmain-content"]/header/div[1]/text()')[0].split('|')[1].replace(' ','').replace('\n','')
   #original source 
    for i in range(1,5):
        source = parse_html.xpath('//*[@id="sidebar-content"]/section[2]/div['+str(i)+']/h4/text()')
        if source and 'Original Source' in source1[0]:
            source_url =parse_html.xpath('//*[@id="sidebar-content"]/section[1]/div['+str(i)+']/a/text()')[0]

    original_source = source_url.replace('\t','').replace('\n','').replace(' ','')

    matrix= [[euid,url, doi, title, description, keywords, funder, journal, category, institution,meeting,date,original_source]]
    file1 = open('D:/Users/zhangz10/ER/metadata_data.csv', 'a+', newline='', encoding='utf-8')
    writer1= csv.writer(file1, delimiter=';')
    writer1.writerows(matrix)
    file1.close()
    doi = title= description= keywords= funder= journal= category= institution= meeting= date=orginal_source= None None

 #full-text     
    all_link= news= image= None
    
    content = soup.find_all('div',class_ = 'ahcenter')[0]
    #content = soup.find_all('div',class_ = 'entry')[0]
    new = content.find_all('p')
    news1 = ''
    for j in range(len(new)):
        con = parse_html.xpath('//*[@id="ahmain-content"]/div[1]/p['+str(j)+']/text()')
        #con = parse_html.xpath('//*[@id="main-content"]/article/div/p['+str(j)+']/text()')
        if con:
            news1 += str(con[0].replace('\t','').replace('\n','').replace('  ',''))+' '            #newscontent
    news = news1
    
    link = content.find_all('a',href=True)
    links = ''
    for i in link:   #['href']#all url do for
        if 'http' in i['href']:
            links += str(i['href'])+ ','      #alllink
    all_link = links
    
    images = ''
    for image in soup.find_all('figure'):
       
        if image.find_all('a'):
            image_id = image.find_all('a')[0]['href']  #//*[@id="main-content"]/article/div[1]/figure/figcaption/p[1]/strong/text()
            image_caption = image.find_all('strong')[0].string #image.find_all('img')[0]['title']#
            if image.find_all('img'):
                image_url = image.find_all('img')[0]['src']
            else:
                image_url = None
            images += str(image_id)+','
            picture = [[euid,image_id,image_caption,image_url]]
            file3 = open('D:/Users/zhangz10/ER/newimage_twitter.csv', 'a+', newline='', encoding='utf-8')
            writer3= csv.writer(file3, delimiter=';').writerows(picture)
            file3.close()
    all_images = images
    
    fulltext = [[euid,news,all_images,all_link]]
    file2 = open('D:/Users/zhangz10/ER/newfulltext_twitter.csv', 'a+', newline='', encoding='utf-8')
    writer2= csv.writer(file2, delimiter=';')
    writer2.writerows(fulltext)
    all_link= news= image= None
    file2.close()


In [127]:
def cpunum(data):
        n_cpu = 10 # get CPU number
        num = len(data) # 数据集样本数量
        #data.n_cpu = n_cpu
        #print('cpu num: {}'.format(n_cpu))
        chunk_size = int(num / n_cpu) # 分摊到每个CPU上的样本数量
        procs = []
        for i in range(0, n_cpu):
            min_i = chunk_size * i
            if i < n_cpu - 1:
                max_i = chunk_size * (i + 1)
            else:
                max_i = num
            digits = [data[min_i: max_i],i,min_i,max_i]
            procs.append(digits) 
            #t = threading.Thread(target=process, args=(digits[0],digits[2],digits[3],"parallel"))
            #t.start()
        return procs

def run(twdata):
    s = time.time()
    #q =Queue()
    threads = []
    m = cpunum(twdata)
    for i in range(10):
        t = threading.Thread(target=process, args=(m[i][0],m[i][2],m[i][3],m[i][1]))
        t.start()
        threads.append(t)
    for thread in threads:
        thread.join()
    #results = []
    #for _ in range(20):
   #     results += q.get()
    t.join()
    e = time.time()
    print("\n应用多进程耗时: %0.2f seconds" % (e - s))
    #return results

def process(data,start,end,name):
    print('执行任务%s (%s)...' % (name, (end-start)))
    s = time.time()
    l1 = []
    for i in range(start,end):
        euid = data['euid'][i]
        url = 'http://www.eurekalert.org/news-releases/'+ str(euid)
        try:
            get_eurekalert_originalurl(euid,url)
        except:
            l1.append(data['euid'][i])
    e = time.time()
    print('任务 %s 运行了 %0.2f seconds.' % (name, (e - s)))

In [ ]:
run(dfurl)